In [ ]:
# import our libraries
import requests
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
import re
import datetime

cn=[]
with open('companyn.txt') as f:
    for line in f:
        cn.append(line.rstrip())
        
for i in range(len(cn)):
    company = cn[i]
    selectedreport = '10-K'
    currentDateTime = datetime.datetime.now()
    date = currentDateTime.date()
    year = date.strftime("%Y")
    year= int(year)
    pryear = year-1
    prryear = year-2
    
    csv = pd.read_csv('final1.tsv', sep='\t',  lineterminator='\n', names=None) 

    base_url = r"https://www.sec.gov"
    csv.columns.values[0] = 'Item'

    companyreport = csv[(csv['Item'].str.contains(company)) & (csv['Item'].str.contains(selectedreport))]

    Filing = companyreport['Item'].str.split('|')
    Filing = Filing.to_list()

    for item in Filing[0]:

        if 'txt' in item:
            report = item
    normal_url = r'https://www.sec.gov/Archives/' + report

    documents_url = normal_url.replace('-','').replace('.txt','/index.json')


    # request the url and decode it.
    a = 0
    while a != 1:

        content = requests.get(documents_url)
        try:
            data = content.json()
            a+=1
        except ValueError:
            a=0


    for file in data['directory']['item']:

        # Grab the filing summary and create a new url leading to the file so we can download it.
        if file['name'] == 'FilingSummary.xml':

            xml_summary = base_url + data['directory']['name'] + "/" + file['name']
    # define a new base url that represents the filing folder. This will come in handy when we need to download the reports.
    base_url = xml_summary.replace('FilingSummary.xml', '')

    # request and parse the content


    a = 0
    while a != 200:

        re = requests.get(xml_summary)
        try:
            content = re.text
            a=re.status_code
        except ValueError:
            a=0
    soup = BeautifulSoup(content, 'html5lib')

    # find the 'myreports' tag because this contains all the individual reports submitted.
    reports = soup.find('myreports')

    # I want a list to store all the individual components of the report, so create the master list.
    master_reports = []

    # loop through each report in the 'myreports' tag but avoid the last one as this will cause an error.
    for report in reports.find_all('report')[:-1]:

        # let's create a dictionary to store all the different parts we need.
        report_dict = {}
        report_dict['name_short'] = report.shortname.text
        report_dict['name_long'] = report.longname.text
        report_dict['position'] = report.position.text
        report_dict['category'] = report.menucategory.text
        report_dict['url'] = base_url + report.htmlfilename.text

        # append the dictionary to the master list.
        master_reports.append(report_dict)

        # print the info to the user.

    listm=['CONSOLIDATED STATEMENTS OF OPERATIONS', 'Consolidated Statements of Operations', 'Consolidated Statements of Income', 'CONSOLIDATED STATEMENTS OF NET INCOME', 'STATEMENTS OF OPERATIONS']
    mrn=[]
    for i in range(len(master_reports)):
        mrn.append(master_reports[i].get('name_short'))
    match = []
    for i in listm:
        if i in mrn:
            match.append(i)

    match.append(mrn[1])
    mr=match[0]

    # create the list to hold the statement urls
    statements_url = []

    for report_dict in master_reports:

        # define the statements we want to look for.
        item1 = mr

        # store them in a list.
        report_list = [item1]

        # if the short name can be found in the report list.
        if report_dict['name_short'] in report_list:

            # print some info and store it in the statements url.
            print('-'*100)
            print(report_dict['name_short'])
            print(report_dict['url'])

            statements_url.append(report_dict['url'])
    # let's assume we want all the statements in a single data set.
    statements_data = []

    # loop through each statement url
    for statement in statements_url:

        # define a dictionary that will store the different parts of the statement.
        statement_data = {}
        statement_data['headers'] = []
        statement_data['sections'] = []
        statement_data['data'] = []

        # request the statement file content
        a = 0
        while a != 200:

            re = requests.get(statement)
            try:
                content = re.text
                a=re.status_code
            except ValueError:
                a=0
        report_soup = BeautifulSoup(content, 'html5lib')

        # find all the rows, figure out what type of row it is, parse the elements, and store in the statement file list.
        for index, row in enumerate(report_soup.table.find_all('tr')):

            # first let's get all the elements.
            cols = row.find_all('td')

            # if it's a regular row and not a section or a table header
            if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
                reg_row = [ele.text.strip() for ele in cols]
                statement_data['data'].append(reg_row)

            # if it's a regular row and a section but not a table header
            elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
                sec_row = cols[0].text.strip()
                statement_data['sections'].append(sec_row)

            # finally if it's not any of those it must be a header
            elif (len(row.find_all('th')) != 0):            
                hed_row = [ele.text.strip() for ele in row.find_all('th')]
                statement_data['headers'].append(hed_row)

            else:            
                print('We encountered an error.')

        # append it to the master list.
        statements_data.append(statement_data)
    # Grab the proper components
    income_header =  statements_data[0]['headers'][1]
    income_data = statements_data[0]['data']

    # Put the data in a DataFrame
    income_df = pd.DataFrame(income_data)

    # Define the Index column, rename it, and we need to make sure to drop the old column once we reindex.
    income_df.index = income_df[0]
    income_df.index.name = 'Category'
    income_df = income_df.drop(0, axis = 1)



    # Get rid of the '$', '(', ')', and convert the '' to NaNs.
    income_df = income_df.replace('[\$,)]','', regex=True )\
                         .replace( '[(]','-', regex=True)\
                         .replace( '', np.nan, regex=True)

    # show the df
    income_df


    # drop the data in a CSV file if need be.
    # income_df.to_csv('income_state.csv')
    df=income_df.loc[income_df.index.isin(['Net income (loss)', 'Net loss', 'Net loss attributable to AAM', 'Net (loss) earnings attributable to Modine', 'Net Income', 'Net income', 'Net income attributable to Aptiv', 'Operating Income', 'Product revenues', 'Revenues (Note 3)', 'Net earnings attributable to BorgWarner Inc.', 'Net Income (Loss)', 'Net sales and operating revenues', '(LOSS) INCOME FROM OPERATIONS', 'NET LOSS ATTRIBUTABLE TO STRATTEC SECURITY CORPORATION', 'NET SALES', 'Net revenues', 'Income from operations', 'Operating (loss) income', 'OPERATING INCOME (LOSS)', 'NET INCOME', 'Consolidated net income', 'Net earnings', 'Revenue', 'Sales', 'Net (loss) income', 'Sales (D)', 'Revenues', 'Net Sales', 'Net sales', 'OPERATING INCOME', 'Operating income', 'NET INCOME ATTRIBUTABLE TO CUMMINS INC.', 'Operating income (loss)','Operating profit'])]
    if df.isnull().any().any() == True:
        df=df.dropna(axis=1)
    else:
        df=df
    df=df.astype(float)
    df['5'] = df.apply(lambda row: round(abs((row.iloc[0]-row.iloc[1])/row.iloc[1]*100), 1), axis=1)
    df['5'] = np.where(df['5'] > 100, '_', df['5'])
    df=df.head(3)
    if company == "LITTELFUSE INC /DE":
        compny = "Littlefuse"
    elif company == "GOODYEAR TIRE & RUBBER CO /OH/":
        compny = "Goodyear"
    else:
        compny = company
    csv_f_path= compny+'.csv'
    df.to_csv(csv_f_path, header=False) 
    df2=income_df.loc[income_df.index.isin(['Research, development and engineering expenses','Research and development expenses','Research and development','Research and development expense','Engineering, research and development','Selling, administrative, and research and development expenses','Engineering, research, and development'])]
    if (len(df2))>0: 
        if df2.isnull().any().any() == True:
            df2=df2.dropna(axis=1)
        else:
            df2=df2
        df2=df2.astype(float)
        df2['5'] = df2.apply(lambda row: round(abs((row.iloc[0]-row.iloc[1])/row.iloc[1]*100), 1), axis=1)
        df2['5'] = np.where(df2['5'] > 100, '_', df2['5'])
        df2=df2.head(1)
        dfi = df2.index[0]
        dfif=dfi.replace(",","")

        as_list = df2.index.tolist()
        idx = as_list.index(dfi)
        as_list[idx] = dfif
        df2.index = as_list

        if company == "LITTELFUSE INC /DE":
            compny = "Littlefuse"
        elif company == "GOODYEAR TIRE & RUBBER CO /OH/":
            compny = "Goodyear"
        else:
            compny = company
        csv2_f_path= compny+'RND.csv'
        df2.to_csv(csv2_f_path, header=False)

    else:
        csv2_f_path= compny+'RND.csv'

    if company=='AMERICAN AXLE & MANUFACTURING HOLDINGS INC':
        company='axl'
    elif company=='Cooper-Standard Holdings Inc.':
        company='0001320461'
    elif company=='GOODYEAR TIRE & RUBBER CO /OH/':
        company='0000042582'
    else:
        company=company
    infile = open(csv_f_path,"r")
    lis=[]
    for line in infile:
        row = line.split(",")
        cat = row[0]
        lis.append(cat)
    lis
    net_sales = ['Net sales', 'Net Sales', 'Sales (D)', 'Revenues', 'Revenue', 'Sales', 'NET SALES', 'Net revenues', 'Net sales and operating revenues', 'Product revenues', 'Revenues (Note 3)']
    operating_income= ['Operating Income', 'Operating profit', '(LOSS) INCOME FROM OPERATIONS', 'Income from operations', 'Operating (loss) income', 'OPERATING INCOME (LOSS)', 'OPERATING INCOME', 'Operating income', 'Operating income (loss)']
    net_income = ['Net income (loss)', 'Net loss', 'Net income', 'Net Income', 'Net (loss) earnings attributable to Modine', 'Net income attributable to Aptiv', 'NET INCOME', 'Net Income (Loss)', 'NET INCOME', 'Consolidated net income', 'Net loss attributable to AAM', 'Net earnings', 'NET LOSS ATTRIBUTABLE TO STRATTEC SECURITY CORPORATION', 'NET INCOME ATTRIBUTABLE TO CUMMINS INC.', 'Net (loss) income', 'Net earnings']
    
    cwd = os.getcwd()
    cmpny=company.upper()
    path=cwd+"/txtf/sec-edgar-filings/"+cmpny+"/"+selectedreport
    directories = os.listdir(path)

    for file in directories:
       a = file
    f_path=cwd+"/txtf/sec-edgar-filings/"+cmpny+"/"+selectedreport+"/"+a+"/filing-details.html"
    m_path = f_path.replace("\\","/")
    file1 = open(m_path,"r")
    def clean_me(html):
        soup = BeautifulSoup(html,'html5lib')
        for s in soup(['script', 'style','td']):
            s.decompose()
        return ' '.join(soup.stripped_strings)
    wh=clean_me(file1)
    wh=wh.replace(u'\xa0',' ')
    wh=wh.replace(u'\u200b','')
    wh=wh.replace("we ", "they ")
    wh=wh.replace("We ", "they ")
    wh=wh.replace("our ", "their ")
    wh=wh.replace("Our ", "Their ")
    wh=wh.replace("$", "USD ")
    wh=wh.replace("also ", "")
    wh=wh.replace("theyre ", "they are")
    wh=wh.replace("theyve ", "they have")
    wh=wh.replace("44 Table of Contents ", " ")
    wh=wh.replace("45 Table of Contents ", " ")
    wh=wh.replace("32 Table of Contents ", " ")
    wh=wh.replace("80 Table of Contents ", " ")
    wh=wh.replace(" Table of Contents ", " ")
    wh=wh.replace("52 Table of Contents ", " ")
    wh2=wh.replace(" • ", ".")
    
    import re
    nsl=[]
    for i in range (len(net_sales)):
        for j in range (len(lis)):

            if lis[j] in net_sales[i]:
                fc=df.iloc[j]
                rcns=fc.iloc[-1]
                nscy=fc.iloc[0]
                nspy=fc.iloc[1]
                if rcns == '0.0':
                    dein = "flat"
                elif nspy < nscy:
                    dein= "increased"
                else:
                    dein = "decreased"
                
                if dein == "increased":
                    inde=['increase','increased','gain','gross','impact','amounts']
                elif dein == "decreased":
                    inde=['decrease','decreased','loss','gross','impact','amounts']
                else:
                    inde=['increase','decrease','increased','decreased','gross','impact','amounts']
                defin=['Net sales for','net sales for','Net sales','net sales','Sales','sales','Total net sales']
                
                li=[]

                for i in range (len(defin)):
                    for j in range (len(inde)):
                        if defin[i] and inde[j] in wh:
                            defi = re.findall(r"(([.][^.]*|\.\s[^.]*\.(?!\s)[^.]*)"+defin[i]+"[^.]*"+inde[j]+"([^.]*\.\s|[^.]*\.(?!\s)[^.]*\.\s))",wh)
                            defis = re.findall(r"(([.][^.]*|\.\s[^.]*\.(?!\s)[^.]*)"+inde[j]+"[^.]*"+defin[i]+"([^.]*\.\s|[^.]*\.(?!\s)[^.]*\.\s))",wh)
                            li.append(defi)
                            li.append(defis)
                    
                li = [ele for ele in li if ele != []]
                li=str(li)
                unwanted = "'\/[]!@#:!*)(&^~,-—"
                fi=li
                # Run for loop with each character of sting and remove unwanted characters
                for char in unwanted:
                    fi = fi.replace(char,"")
                fi=re.sub("\s\s+", " ", fi)
                y= re.findall(r"([^.]*(due to a|driven by|impacted|reduction|slowdown|effects|effect|partially offset|reflect|totaled|declined|foreign currencies|escalat|acquisition|primarily due|attributable|due to volumes|driven primarily|affect|as a result)([^.]*\.\s|[^.]*\.(?!\s)[^.]*\.\s))",fi)
                if y == None: 
                    y[0]=("")
                unwanted = "'/\[]!@#:!*)(&^~,-—"
                f=str(y)
                # Run for loop with each character of sting and remove unwanted characters
                for char in unwanted:
                    f = f.replace(char,"")
                f=re.sub("\s\s+", " ", f)
                f=f.split(". ")
                fn=f[0].split(";")
                im=fn
                im
                

                if im==['']:
                    nsl.append("The Company's Net Sales in the fiscal year ended December 31, "+str(pryear)+" was "+dein+" by "+str(abs(float(rcns)))+"% as compared to "+str(prryear)+" from USD "+str(nspy)+" to USD "+str(nscy)+" million.")
                else:
                    nsl.append("The Company's Net Sales in the fiscal year ended December 31, "+str(pryear)+" was "+dein+" by "+str(abs(float(rcns)))+"% as compared to "+str(prryear)+" from USD "+str(nspy)+" to USD "+str(nscy)+" million. "+im[0]+".")
            else:
                nsl.append("")
    fns=list(dict.fromkeys(nsl))
    fn=list(filter(None, fns))
    if len(fn) == 0:
        fnets=""
        print(fnets)
    else:
        fnets=fn[0]
        print(fnets)
    oil=[]

    for i in range (len(operating_income)):
        for j in range (len(lis)):

            if lis[j] in operating_income[i]:
                fc=df.iloc[j]
                rcoi=fc.iloc[-1]
                oicy=fc.iloc[0]
                oipy=fc.iloc[1]
                if rcoi == '0.0':
                    deino = "flat"
                if rcoi == '_':
                    deino = "decreased"
                    rcoi = "NAN"
                elif oipy < oicy:
                    deino= "increased"
                else:
                    deino = "decreased"
                
                if deino == "increased":
                    de=['increase','increased','gain','change','expected','Earning','computed','up','impact','gross','expense','amounts']
                elif deino == "decreased":
                    de=['decrease','decreased','loss','change','down','expected','Earning','computed','impact','gross','expense','amounts']
                else:
                    de=['increase','decrease','increased','decreased','change', 'changes','gain','loss','up','down','expected','Earning','computed','impact','gross','expense','amounts']
                defin2=['operating income ', 'Operating Income ', 'Operating income ', 'OPERATING INCOME', 'operating loss', 'operating expenses']
                
                li2=[]

                for i in range (len(defin2)):
                    for j in range (len(de)):
                        if defin2[i] and de[j] in wh:
                            defi2 = re.findall(r"(([.][^.]*|\.\s[^.]*\.(?!\s)[^.]*)"+defin2[i]+"[^.]*"+de[j]+"([^.]*\.\s|[^.]*\.(?!\s)[^.]*\.\s))",wh)
                            defis2 = re.findall(r"(([.][^.]*|\.\s[^.]*\.(?!\s)[^.]*)"+de[j]+"[^.]*"+defin2[i]+"([^.]*\.\s|[^.]*\.(?!\s)[^.]*\.\s))",wh)
                            li2.append(defi2)
                            li2.append(defis2)
                        
                li2 = [ele for ele in li2 if ele != []]
                li2=str(li2)
                unwanted = "'/\[]!@#;:!*)(&^~,-—"
                fi2=li2
                # Run for loop with each character of sting and remove unwanted characters
                for char in unwanted:
                    fi2 = fi2.replace(char,"")
                fi2=re.sub("\s\s+", " ", fi2)
                y2= re.findall(r"([^.]*(amount|result|factor|due to the|reduce|up|down|gain|profit |earning|compared|caused|expenses|despite|impact|market|share|positive|negative|losses|growth|attribute to|maximum|outperform|performance|expand|achieve|record|forecast|forecasted|carryforward)([^.]*\.\s|[^.]*\.(?!\s)[^.]*\.\s))",fi2)
                if y2 == None:
                    y2[0]=("")
                unwanted = "'/\[]!@#;:!*)(&^~,-—"
                f2=str(y2)
                # Run for loop with each character of sting and remove unwanted characters
                for char in unwanted:
                    f2 = f2.replace(char,"")
                f2=re.sub("\s\s+", " ", f2)
                f2=f2.split(". ")
                fn2=f2[0].split(" - ")
                fo2=fn2[0].split(";")
                imp=fn2

                if imp==['']:
                    oil.append("The Comapany had an Operating income of USD "+str(oicy)+" in the fiscal year ended December 31, "+str(pryear) +" "+ deino+" by "+str(abs(float(rcoi)))+"% compared to an Operating income in the "+str(prryear)+" of USD "+str(oipy)+".")
                else:
                    oil.append("The Comapany had an Operating income of USD "+str(oicy)+" in the fiscal year ended December 31, "+str(pryear) +" "+ deino+" by "+str(abs(float(rcoi)))+"% compared to an Operating income in the "+str(prryear)+" of USD "+str(oipy)+". "+imp[0]+".")

            else:
                oil.append("")
    foi=list(dict.fromkeys(oil))
    fo=list(filter(None, foi))
    if len(fo) == 0:
        foper=""
        print(foper)
    else:
        foper=fo[0]
        print(foper)

    nil=[]
    for i in range (len(net_income)):
        for j in range (len(lis)):

            if lis[j] in net_income[i]:
                fc3=df.iloc[j]
                rcni=fc3.iloc[-1]
                nicy=fc3.iloc[0]
                nipy=fc3.iloc[1]
                if rcni == '0.0':
                    deinn = "flat"
                elif rcni == '_':
                    deinn = "decreased"
                    rcni = "NAN"
                elif nipy < nicy:
                    deinn= "increased"
                else:
                    deinn = "decreased"
                
                if deinn == "increased":
                    den=['increase','increased','gain','change','expected','Earning','computed','up','impact','gross','amounts']
                elif deinn == "decreased":
                    den=['decrease','decreased','loss','change','down','expected','Earning','computed','impact','gross','amounts']
                else:
                    den=['increase','decrease','increased','decreased','change', 'changes','gain','loss','up','down','expected','Earning','computed','impact','gross','amounts']
                defin6=['Net Income', 'net income ', 'Net income ', 'NET INCOME', 'Net income loss', 'net earning', 'Net earning']
                
                li6=[]

                for i in range (len(defin6)):
                    for j in range (len(den)):
                        if defin6[i] and den[j] in wh:
                            defi6 = re.findall(r"(([.][^.]*|\.\s[^.]*\.(?!\s)[^.]*)"+defin6[i]+"[^.]*"+den[j]+"([^.]*\.\s|[^.]*\.[^.]*\.\s))",wh)
                            defis6 = re.findall(r"(([.][^.]*|\.\s[^.]*\.(?!\s)[^.]*)"+den[j]+"[^.]*"+defin6[i]+"([^.]*\.\s|[^.]*\.[^.]*\.\s))",wh)
                            li6.append(defi6)
                            li6.append(defis6)
                        
                li6 = [ele for ele in li6 if ele != []]
                li6=str(li6)
                unwanted = "'/\[]!@#;:!*)(&^~,"
                fi6=li6
                # Run for loop with each character of sting and remove unwanted characters
                for char in unwanted:
                    fi6 = fi6.replace(char,"")
                fi6=re.sub("\s\s+", " ", fi6)
                y6= re.findall(r"([^.]*(result|factor|affect|due to the|reduce|up|down|gain|profit|earning |compared|caused|expense|composed|despite|impact|market|share|positive|primarily due to|negative|net loss|net loss|losses|growth|attribute to|maximum|outperform|performance|expand|achieve|record|forecast|forecasted|outstand|loss|gain|reflect)([^.]*\.\s|[^.]*\.(?!\s)[^.]*\.\s|[^.]*\.(?!\s)[^.]*\.(?!\s)[^.]*\.(?!\s)[^.]*\.\s|[^.]*\.(?!\s)[^.]*\.(?!\s)[^.]*\.\s))",fi6)
                if y6 == None:
                    y6[0]=("")
                unwanted = "'/\[]!@#;:!*)(&^~,"
                f6=str(y6)
                # Run for loop with each character of sting and remove unwanted characters
                for char in unwanted:
                    f6 = f6.replace(char,"")
                f6=re.sub("\s\s+", " ", f6)
                f6=f6.split(". ")
                f6=f6[0].split(" - ")
                fn6=f6[0].split(";")
                impn=fn6

                if impn==['']:
                    nil.append("The Comapany had a Net income of USD "+str(nicy)+" million in the fiscal year ended December 31, "+str(pryear)+" "+ deinn+" by "+str(abs(float(rcni)))+"% compared to an net income in the "+str(prryear)+" of USD "+str(nipy)+".")
                else:
                    nil.append("The Comapany had a Net income of USD "+str(nicy)+" million in the fiscal year ended December 31, "+str(pryear)+" "+ deinn+" by "+str(abs(float(rcni)))+"% compared to an net income in the "+str(prryear)+" of USD "+str(nipy)+". "+impn[0]+".")

            else:
                nil.append("")

    fni=list(dict.fromkeys(nil))
    fn2=list(filter(None, fni))
    if len(fn2) == 0:
        fneti=""
        print(fneti)
    else:
        fneti=fn2[0]
        print(fneti)


    defin3=[' Research and development ', ' Research and Development ', 'RESEARCH AND DEVELOPMENT']

    li3=[]

    for i in range (len(defin3)):
        if defin3[i] in wh:
                defi3 = re.findall(r"([^.]*"+defin3[i]+"([^.]*\.\s|[^.]*\.(?!\s)[^.]*\.\s))",wh2)
                li3.append(defi3)
        
    li3 = [ele for ele in li3 if ele != []]
    li3=str(li3)
    unwanted = "'/\[]!@#;:!*)(^~,-—"
    fi3=li3
    # Run for loop with each character of sting and remove unwanted characters
    for char in unwanted:
        fi3 = fi3.replace(char,"")
    fi3=re.sub("\s\s+", " ", fi3)
    import re
    y3= re.findall(r"([^.]*(COST|cost|Cost|product|patent|engineering|technology|autonomous|electrification|electrified|connected|connectivity|expense |cost|laboratory|estimate|quality|invest|future|improve|requirements|performance|component|activities research|solution|manufaturing|distribution)([^.]*\.\s|[^.]*\.[^.]*\.\s))",fi3)
    if y3 == None:
        y3[0]=("")
    unwanted = "'/\[]!@#;:!*)(&^~,-—"
    f3=str(y3)
    # Run for loop with each character of sting and remove unwanted characters
    for char in unwanted:
        f3 = f3.replace(char,"")
    f3=re.sub("\s\s+", " ", f3)
    f3=f3.split(". ")
    g3=f3[0].split("•")
    if g3==['']:
        frnd=""
        print(frnd)
    else:
        frnd="In the year fiscal year ended December,31 "+str(pryear)+", "+g3[0]+"."
        print(frnd)
    defin4=['Capital expenditure', 'Capital Expenditure', 'CAPITAL EXPENDITURE', 'capital expenditure']

    li4=[]

    for i in range (len(defin4)):
        if defin4[i] in wh:
                defi4 = re.findall(r"([^.]*"+defin4[i]+"([^.]*\.\s|[^.]*\.(?!\s)[^.]*\.\s))",wh2)
                li4.append(defi4)
        
    li4 = [ele for ele in li4 if ele != []]
    li4=str(li4)
    unwanted = "'/\[]!@#;:!*)(&^~,-—"
    fi4=li4
    # Run for loop with each character of sting and remove unwanted characters
    for char in unwanted:
        fi4 = fi4.replace(char,"")
    fi4=re.sub("\s\s+", " ", fi4)
    fi4
    import re
    y4= re.findall(r"([^.]*(capex|environment|land|site|manufacturing|plant|estimate|increase|decrease|spend|software|invest|rate|target|product line|capacity expansion|expect|area|acquisition|financed|expenditure |administrative|corporate)([^.]*\.\s|[^.]*\.[^.]*\.\s))",fi4)
    if y4 == None:
        y4[0]=("")
    unwanted = "'/\[]!@#;:!*)(&^~,-—"
    f4=str(y4)
    # Run for loop with each character of sting and remove unwanted characters
    for char in unwanted:
        f4 = f4.replace(char,"")
    f4=re.sub("\s\s+", " ", f4)
    f4=f4.split(". ")
    if f4==['']:
        fce=""
        print(fce)
    else:
        fce="In the year fiscal year ended December,31 "+str(pryear)+", "+f4[0]+"."
        print(fce)

    defin5=['Outlook', 'OUTLOOK']

    li5=[]

    for i in range (len(defin5)):
        if defin5[i] in wh:
                defi5 = re.findall(r"([^.]*"+defin5[i]+"([^.]*\.\s|[^.]*\.(?!\s)[^.]*\.\s))",wh2)
                li5.append(defi5)
        
    li5 = [ele for ele in li5 if ele != []]
    li5=str(li5)
    unwanted = "'/\[]!@#;:!*)(&^~,-—"
    fi5=li5
    # Run for loop with each character of sting and remove unwanted characters
    for char in unwanted:
        fi5 = fi5.replace(char,"")
    fi5=re.sub("\s\s+", " ", fi5)
    y5= re.findall(r"([^.]*(forecast |estimate |commit |forecasted |plan |figure |increase |decrease |limited |impact |financial |performance |innovative |future |predict |anticipate |risk |overall |customer demand |persist |supply chain impact |operation |shutdown |materials |securities |shortage |demand |government |requirement |distribution |plant closure |strong |weak |strengthen |improve |decline |long-term)[^.]*\.)",fi5)
    if y5 == None:
        y5[0]=("")
    unwanted = "'/\[]!@#;:!*)(&^~,-—"
    f5=str(y5)
    # Run for loop with each character of sting and remove unwanted characters
    for char in unwanted:
        f5 = f5.replace(char,"")
    f5=re.sub("\s\s+", " ", f5)
    f5=f5.split(". ")
    g5=f5[0]
    if g5=='':
        fout=""
        print(fout)
    else:
        fout="In the year fiscal year ended December,31 "+str(pryear)+", "+g5+"."
        print(fout)
    cate=[]
    if fnets == "" :
        cate
    else:
        cate.append("Net Sales")   
    if foper == "" :
        cate
    else:
        cate.append("Operating Income")
    if fneti == "" :
        cate
    else:
        cate.append("Net Income")

    cate
    dfc = pd.read_csv(csv_f_path, header = None)
    rc=dfc.iloc[:,-1]
    for i in range(len(dfc)):
        dfc=dfc.replace(to_replace = dfc.iloc[i][0],value = cate[i])
        if dfc.iloc[i][1] < dfc.iloc[i][2]: 
            if rc[i]!='_':
                dfc=dfc.replace(to_replace = rc[i],value = "("+str(abs(float(rc[i])))+")")
    dfc.to_csv(csv_f_path,index=False,header=None)
    rndh=["Research and Development"]
    if (len(df2))>0:
        dfc2 = pd.read_csv(csv2_f_path, header = None)
        rc2=dfc2.iloc[:,-1]
        for i in range(len(dfc2)):
            dfc2=dfc2.replace(to_replace = dfc2.iloc[i][0],value = rndh[i])
            if dfc2.iloc[i][1] < dfc2.iloc[i][2]: 
                if rc2[i]=='_':
                    rc2[i]=rc2[i]
                else:
                    dfc2=dfc2.replace(to_replace = rc2[i],value = "("+str(abs(float(rc2[i])))+")")
        dfc2.to_csv(csv2_f_path,index=False,header=None)



    import sys
    html_f_path =compny+".html"
    # Saving the reference of the standard output
    original_stdout = sys.stdout    
    with open(html_f_path, 'w') as f:
        if len(df.columns) == 4:
            sys.stdout = f 
            print('<table>')
            print("<tbody>")
            print("<tr>")
            print('<td>')
            print('<h4>Financial Overview</h4>')# begin the table
            print('</td>')
            print('<td class="right">(in million USD)</td>')
            print('</tr>')
            print('</tbody>')
            print('</table>')
            print('<table class="sheet">')
            print('<tbody>')
            print('<tr class="center">')
            print('<td width="40%">&nbsp;</td>')
            print('<td width="10%">FY ended Dec. 31, '+str(pryear)+'</td>')
            print('<td width="10%">FY ended Dec. 31, '+str(prryear)+'</td>')
            #print('<td width="10%">FY ended Dec. 31, '+str(prryear-1)+'</td>')
            print('<td width="10%">Rate of change (%)</td>')
            print("</tr>")

            infile = open(csv_f_path,"r")

            for line in infile:
                row = line.split(",")
                Category = row[0]
                FY20 = row [1]
                FY19 = row[2]
                #FY18 = row[3]
                Rateofchange = row[4]
                #Factors = row[5]

                print("<tr>")
                print('<td>%s</td>' % Category)
                print('<td class="right">%s</td>' % FY20)
                print('<td class="right">%s</td>' % FY19)
                #print('<td class="right" width="86">%s</td>' % FY18)
                print('<td class="right">%s</td>' % Rateofchange)
                #print('<td width="238">%s</td>' % Factors)
                print("</tr>")
            print("</tbody>")
            # end the table
            print("</table>")


        else:
            sys.stdout = f 
            print('<table>')
            print("<tbody>")
            print("<tr>")
            print('<td>')
            print('<h4>Financial Overview</h4>')# begin the table
            print('</td>')
            print('<td class="right">(in million USD)</td>')
            print('</tr>')
            print('</tbody>')
            print('</table>')
            print('<table class="sheet">')
            print('<tbody>')
            print('<tr class="center">')
            print('<td width="40%">&nbsp;</td>')
            print('<td width="10%">FY ended Dec. 31, '+str(pryear)+'</td>')
            print('<td width="10%">FY ended Dec. 31, '+str(prryear)+'</td>')
            #print('<td width="10%">FY ended Dec. 31, '+str(prryear-1)+'</td>')
            print('<td width="10%">Rate of change (%)</td>')
            print("</tr>")

            infile = open(csv_f_path,"r")

            for line in infile:
                row = line.split(",")
                Category = row[0]
                FY20 = row [1]
                FY19 = row[2]
                #FY18 = row[3]
                Rateofchange = row[3]
                #Factors = row[5]

                print("<tr>")
                print('<td>%s</td>' % Category)
                print('<td class="right">%s</td>' % FY20)
                print('<td class="right">%s</td>' % FY19)
                #print('<td class="right">%s</td>' % FY18)
                print('<td class="right">%s</td>' % Rateofchange)
                #print('<td width="238">%s</td>' % Factors)
                print("</tr>")
            print("</tbody>")
            # end the table
            print("</table>")


        if fnets == "" :
            print("")
        else:
            print('<p><strong>Factors</strong><br />1)&nbsp;<span style="text-decoration: underline;">Net Sales</span><br />-' +fnets)
        if foper == "" :
            print("")
        else:
            print('<br /><br />2) <span style="text-decoration: underline;">Operating Income</span><br />-' +foper)
        if fneti == "" :
            print("")
        else:
            print('<br /><br />3)&nbsp;<span style="text-decoration: underline;">Net Income</span><br />-'+fneti)
        print('<br /><br /></p>')


        if frnd == "" :
            print("")
        else:
            print('<h4>Research and Development</h4>')     

            if (len(df2))>0:

                print('<table class="sheet">')
                print('<tbody>')
                print('<tr class="center">')
                print('<td width="40%">&nbsp;</td>')
                print('<td width="10%">FY ended Dec. 31, '+str(pryear)+'</td>')
                print('<td width="10%">FY ended Dec. 31, '+str(prryear)+'</td>')
                #print('<td width="10%">FY ended Dec. 31, '+str(prryear-1)+'</td>')
                print('<td width="10%">Rate of change (%)</td>')
                print("</tr>")

                infile2 = open(csv2_f_path,"r")

                for line in infile2:
                    row = line.split(",")
                    Category = row[0]
                    FY20 = row [1]
                    FY19 = row[2]
                    #FY18 = row[3]
                    Rateofchange = row[4]
                    #Factors = row[5]

                    print("<tr>")
                    print('<td>%s</td>' % Category)
                    print('<td class="right">%s</td>' % FY20)
                    print('<td class="right">%s</td>' % FY19)
                    #print('<td class="right">%s</td>' % FY18)
                    print('<td class="right">%s</td>' % Rateofchange)
                    #print('<td width="238">%s</td>' % Factors)
                    print("</tr>")
                print("</tbody>")
                # end the table
                print("</table>")

            else:
                print("")

            print('<p>-'+frnd+'<br /><br /></p>')



        if fce == "" :
            print("")
        else:
            print('<h4>Capital Expenditure</h4>')
            print('<p>-'+fce+'<br /><br /></p>')

        if fout == "" :
            print("")
        else:
            print('<h4>Outlook</h4>')
            print('<p>-'+fout+'<br /><br /></p>')

        sys.stdout = original_stdout 